In [1]:
#Installing the utilities provided by the organizers of the shared task ""
!git clone https://github.com/dkorenci/pan-clef-2024-oppositional.git

Cloning into 'pan-clef-2024-oppositional'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 80 (delta 34), reused 72 (delta 26), pack-reused 0
Receiving objects: 100% (80/80), 3.71 MiB | 24.83 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
!bash /content/pan-clef-2024-oppositional/generate_requirements.sh

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 4.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=837d1c52478e8b16c4062a035e4b0b5b7aec62c50d0ad38bc570edc5dc273b03
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: mistune
    Found existing installation: mistune 0.8.4
    Uninstalling mistune-0.8.4:
      Successfully uninstalled mistune-0.8.4
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
  Attempting uninstall: nbconver

In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [6]:
import os, sys
sys.path.append("/content/pan-clef-2024-oppositional/")
TRAIN_DATASET_ES="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/training/dataset_oppositional/dataset_es_train.json"
TRAIN_DATASET_EN="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/training/dataset_oppositional/dataset_en_train.json"
TEST_DATASET_EN ="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/test/dataset_oppositional_test_nolabels/dataset_en_official_test_nolabels.json"
TEST_DATASET_ES ="/content/drive/MyDrive/LNR/LNR 2024/Dataset-Oppositionl/test/dataset_oppositional_test_nolabels/dataset_en_official_test_nolabels.json"

In [7]:
from data_tools.dataset_loaders import load_dataset_classification
texts_es, labels_es, ids_es = load_dataset_classification("es", string_labels=False, positive_class='conspiracy')
texts_en, labels_en, ids_en = load_dataset_classification("en", string_labels=False, positive_class='conspiracy')


Loading official JSON es dataset
Loading official JSON en dataset


In [12]:
#Creating a toy representation for the training dataset in both languages.
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'word', max_features=4000, lowercase=True)
X_es = vectorizer.fit_transform(texts_es.tolist())
vectorizer.get_feature_names_out()
X_es= X_es.toarray()
Y_es= np.asarray(labels_es.tolist())

vectorizer = CountVectorizer(analyzer = 'word', max_features=4000, lowercase=True)
X_en = vectorizer.fit_transform(texts_en.tolist())
vectorizer.get_feature_names_out()
X_en= X_en.toarray()
Y_en= np.asarray(labels_en.tolist())

In [13]:
#EXAMPLE 1
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn import svm
# Here, we are dividing the trainig set in two subsets one for training and other for validation.
# In practice, X_en and X_es represent the matrices with text representations from data provided, and Y_en and Y_es represent the corresponding labels of either critical or conspiracy texts.
X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

#Using the default parameters
clf_en = svm.SVC()
clf_en.fit(X_en_train, y_en_train)
predicted = clf_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)

#Using the default parameters
clf_es = svm.SVC()
clf_es.fit(X_es_train, y_es_train)
predicted = clf_es.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)

Predicted	-->	 [0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0
 1 0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0
 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0
 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0
 1 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0
 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

In [14]:
#EXAMPLE 2
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

#Using the default parameters
clf_en = LogisticRegression()
clf_en.fit(X_en_train, y_en_train)
predicted = clf_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)


#Using the default parameters
clf_es= LogisticRegression()
clf_es.fit(X_es_train, y_es_train)
predicted = clf_es.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)




/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Predicted	-->	 [0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0
 1 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0
 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0
 1 1 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0
 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
#EXAMPLE 3
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

clf_en = DecisionTreeClassifier()
clf_en.fit(X_en_train, y_en_train)
predicted = clf_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)

clf_es = DecisionTreeClassifier()
clf_es.fit(X_es_train, y_es_train)
predicted = clf_es.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)


Predicted	-->	 [0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0
 1 0 1 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0
 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0
 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0
 1 1 0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0
 1 1 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

In [16]:
#EXAMPLE 4
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

#Using the default parameters
clf_en = MLPClassifier(random_state=1, max_iter=300)
clf_en.fit(X_en_train, y_en_train)
predicted = clf_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)


#Using the default parameters
clf_es = MLPClassifier(random_state=1, max_iter=300)
clf_es.fit(X_es_train, y_es_train)
predicted = clf_es.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)




Predicted	-->	 [1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0
 1 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0
 1 1 0 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 0 0 1 1 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1
 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0
 0 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1
 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0
 1 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0
 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

In [18]:
#EXAMPLE 5
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

#Here you can define the training and test sets for the shared task
X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

# create the base models, Logistic Regression, Decision Tree, and Support Vector Machine
model1 = LogisticRegression()
model2 = DecisionTreeClassifier()
model3 = SVC()
# create the voting ensemble
ensemble_en = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('svm', model3)], voting='hard')
# train the ensemble
ensemble_en.fit(X_en_train, y_en_train)
# Evaluate the ensemble
predicted=ensemble_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)


# create the base models, Logistic Regression, Decision Tree, and Support Vector Machine
model1 = LogisticRegression()
model2 = DecisionTreeClassifier()
model3 = SVC()
# create the voting ensemble
ensemble_es = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('svm', model3)], voting='hard')
# train the ensemble
ensemble_es.fit(X_es_train, y_es_train)
# Evaluate the ensemble
predicted=ensemble_en.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Predicted	-->	 [0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0
 1 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 1 1 0 1 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0
 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0
 1 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0
 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Predicted	-->	 [1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
GroundTruth	-->	 [0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0
 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0
 0 0 1

In [19]:
#EXAMPLE 6
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
#Here you can define the training and test sets for the shared task
X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

ensemble_en =BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10)
# train the ensemble
ensemble_en.fit(X_en_train, y_en_train)
# evaluate the ensemble
ensemble_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)

ensemble_es =BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10)
# train the ensemble
ensemble_es.fit(X_es_train, y_es_train)
# evaluate the ensemble
ensemble_es.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Predicted	-->	 [1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Predicted	-->	 [1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
GroundTruth	-->	 [0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0
 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0
 0 0 1

In [23]:
#EXAMPLE 8
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
#Here you can define the training and test sets for the shared task
X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

ensemble_en =RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
# train the ensemble
ensemble_en.fit(X_en_train, y_en_train)
# evaluate the ensemble
ensemble_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)

ensemble_es =RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
# train the ensemble
ensemble_es.fit(X_es_train, y_es_train)
# evaluate the ensemble
ensemble_es.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)

Predicted	-->	 [1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

In [25]:
#EXAMPLE  9
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression


X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

# Create the boosting classifier
ensemble_en=AdaBoostClassifier(base_estimator=LogisticRegression(),n_estimators=10)
# Train the boosting classifier on the training data
ensemble_en.fit(X_en_train, y_en_train)
# Evaluate the boosting classifier on the testing data
predicted = ensemble_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)

# Create the boosting classifier
ensemble_es=AdaBoostClassifier(base_estimator=LogisticRegression(),n_estimators=10)
# Train the boosting classifier on the training data
ensemble_es.fit(X_es_train, y_es_train)
# Evaluate the boosting classifier on the testing data
predicted = ensemble_es.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Predicted	-->	 [0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0
 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 1 0 0 1 0 1 0 1 0
 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Predicted	-->	 [0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 0
 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1
 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0]
GroundTruth	-->	 [0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0
 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0
 0 0 1

In [28]:
#EXAMPLE 10
!pip install xgboost
import xgboost as xgb
from sklearn.metrics import matthews_corrcoef


X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

# Convert data to DMatrix format
dtrain_en = xgb.DMatrix(X_en_train, label=y_en_train)
dtest_en = xgb.DMatrix(X_en_test, label=y_en_test)

# Set XGBoost parameters
params = {
   'max_depth': 3,
   'eta': 0.1,
   'objective': 'binary:logistic',
   'eval_metric': 'auc'
}

# Train the model
num_rounds = 100
bst_en = xgb.train(params, dtrain_en, num_rounds)
# Make predictions on the test set
predicted = bst_en.predict(dtest_en)
# Evaluate the model
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
predicted = np.asarray([round(value) for value in predicted])
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)


# Convert data to DMatrix format
dtrain_es = xgb.DMatrix(X_es_train, label=y_es_train)
dtest_es = xgb.DMatrix(X_es_test, label=y_es_test)

# Set XGBoost parameters
params = {
   'max_depth': 3,
   'eta': 0.1,
   'objective': 'binary:logistic',
   'eval_metric': 'auc'
}

# Train the model
num_rounds = 100
bst_es = xgb.train(params, dtrain_es, num_rounds)
# Make predictions on the test set
predicted = bst_es.predict(dtest_es)
# Evaluate the model
print("Predicted\t-->\t", predicted)
predicted = np.asarray([round(value) for value in predicted])
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)

Predicted	-->	 [0.18434063 0.2241836  0.11923677 0.20737405 0.06573689 0.6290595
 0.3864707  0.8867527  0.20124248 0.8936915  0.58740264 0.82706803
 0.06040424 0.17905107 0.12950625 0.04967542 0.15846328 0.12672687
 0.8337534  0.68943125 0.698328   0.23992816 0.58177817 0.7861007
 0.19562127 0.42405805 0.04027979 0.16963732 0.20536499 0.17955299
 0.8912415  0.32728812 0.18227866 0.2047077  0.03933467 0.4190578
 0.13323498 0.8044822  0.24382278 0.2352227  0.17350385 0.8071747
 0.7921219  0.10891214 0.13861509 0.13840972 0.6350267  0.1721873
 0.13381204 0.9004269  0.25133812 0.7789721  0.52266824 0.14751948
 0.10433385 0.25791356 0.18227866 0.9696572  0.9612506  0.4532724
 0.30484995 0.8527959  0.14126456 0.14586338 0.32341772 0.80587846
 0.35523322 0.7968726  0.07525552 0.14754106 0.1666345  0.1969439
 0.06933396 0.1480879  0.96226704 0.80055255 0.30354327 0.50533646
 0.8166155  0.27508998 0.692077   0.11307254 0.11236033 0.14126456
 0.7544328  0.31237575 0.86004436 0.17212613 0.8451585

In [29]:
#EXAMPLE 11
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


X_en_train, X_en_test, y_en_train, y_en_test = train_test_split(X_en, Y_en, test_size=0.1, random_state=1234)
X_es_train, X_es_test, y_es_train, y_es_test = train_test_split(X_es, Y_es, test_size=0.1, random_state=1234)

# create the base models,K-NNeighbors, Logistic Regression, Decision Tree, and Support Vector Machine
base_models_en = [('dt', DecisionTreeClassifier()), ('svm', SVC()), ('knn', KNeighborsClassifier())]
# Create the meta-model
meta_model_en = LogisticRegression()
# Create the stacking classifier
ensemble_en = StackingClassifier(estimators=base_models_en, final_estimator=meta_model_en)
# Train the stacking classifier on the training data
ensemble_en.fit(X_en_train, y_en_train)
# Evaluate the stacking classifier on the testing data
predicted = ensemble_en.predict(X_en_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_en_test)
quality=matthews_corrcoef(y_en_test, predicted)
print("MCC for English:", quality)

base_models_es = [('dt', DecisionTreeClassifier()), ('svm', SVC()), ('knn', KNeighborsClassifier())]
# Create the meta-model
meta_model_es = LogisticRegression()
# Create the stacking classifier
ensemble_es = StackingClassifier(estimators=base_models_es, final_estimator=meta_model_es)
# Train the stacking classifier on the training data
ensemble_es.fit(X_es_train, y_es_train)
# Evaluate the stacking classifier on the testing data
predicted = ensemble_es.predict(X_es_test)
print("Predicted\t-->\t", predicted)
print("GroundTruth\t-->\t", y_es_test)
quality=matthews_corrcoef(y_es_test, predicted)
print("MCC for Spanish:", quality)


Predicted	-->	 [0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0
 1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 1 1 0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1
 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0
 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0
 1 1 0 0 0 1 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0
 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
GroundTruth	-->	 [0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 1 1

In [32]:
#EXAMPLE 12
from sklearn import svm
from sklearn.metrics import matthews_corrcoef, make_scorer
mcc_scorer = make_scorer(matthews_corrcoef, sample_weight=None)
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(estimator=svm.SVC(), param_grid={'C': [2,3,4,5,10], 'kernel': ('linear', 'rbf')}, scoring=mcc_scorer, n_jobs=-1, verbose=1)
clf.fit(X_en, Y_en)
#Obtaining the best hyper-parameters
print(clf.best_params_)
#Obtain the best classifier
print(clf.best_estimator_)

{'C': 5, 'kernel': 'rbf'}
SVC(C=5)


In [33]:
#EXAMPLE 13
#- uniform: uniform distribution in: [loc, loc + scale]
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import matthews_corrcoef, make_scorer
mcc_scorer = make_scorer(matthews_corrcoef, sample_weight=None)
from scipy.stats import uniform
logistic = LogisticRegression()
distributions = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])
clf = RandomizedSearchCV(logistic, distributions, scoring=mcc_scorer, n_jobs=-1, verbose=1)
clf.fit(X_es, Y_es)
print(clf.best_params_)
print(clf.best_estimator_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

{'C': 1.7405868128518316, 'penalty': 'l2'}
LogisticRegression(C=1.7405868128518316)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
